# Vectorization of Samples
To conserve resources and prepare the samples for final ingest, the last thing we need to do is turn all of the samples into vectors. This will be done using a custom tokenizer similar to that in tokenization.ipynb in MalDroid_feature_engineering repo. It will take the sample.apk.json files as a string and convert them to index ints. Due to the large size of our training dataset and the potential for the future computation of additional n-grams, performance and reusability will be optimized. The vocabulary.txt file from the aforementioned repo has been copied to local.
## Algorithm Process (for sample in samples; given: regex delimiter from vocab)
1. Fetch sample and load as dict, extract list of behaviors
2. Call multisort function, ordering by id then ts 
3. For behavior in sorted list, drop unused keys
4. Cast each dict in sorted list to string and add to str list of behaviors
5. Initalize vector with SOA index, For behavior in list, call tokenizer function passing vector: 
6. Append vector with SOB index, init prev_matchEnd=0
7. For match in finditer(delim, behavior):
8. If match.start() - prev_matchEnd > 2, append vector with UNK index
9. Append vector with match.lastgroup set prev_matchEnd to match.end(), next match
10. Append vector with EOB index, return vector, next behavior
11. Append vector with EOA index, save to .tfrecord under *class*/*hash*.tfrecord, next sample

In [1]:
import json
import tensorflow as tf
import numpy as np
import ast
import re
import time
import timeit
from statistics import mean

C:\ProgramData\Anaconda3\envs\MITRE_modeling\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\MITRE_modeling\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\MITRE_modeling\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\MITRE_modeling\l

In [2]:
with open('vocabulary.txt') as vocab_path:
    vocab_file = vocab_path.read()

vocab = ast.literal_eval(vocab_file)

token_specification = []
delimiter_list = []

for packed_items in vocab.items():
    index, value = packed_items
    delimiter, literal = value
    if delimiter != None:
        delimiter_list.append(str(delimiter))
        token_specification.append(('I' + str(index), delimiter))

regex_pattern = '|'.join('(?P<{0}>{1})'.format(name, delim) for name, delim in token_specification)
# calling re.compile() on the pattern could lead to increased performance, but causes issues with backslashes, therefore it is omitted

In [3]:
def sort_behaviors(raw_behaviors):
    raw_behaviors.sort(key = lambda x: x['low'][0]['id'])
    raw_behaviors.sort(key = lambda x: float(x['low'][0]['ts']))
    return raw_behaviors

In [4]:
def strip_unused_keys(behavior_list, unused_keys):
    behavior_index = 0
    for behavior in behavior_list:
        behavior = {key: value for key, value in behavior.items() if key not in unused_keys}
        sub_behavior_index = 0
        for sub_behavior in behavior['low']:
            sub_behavior = {sub_key: sub_value for sub_key, sub_value in sub_behavior.items() if sub_key not in unused_keys}
            behavior['low'][sub_behavior_index] = sub_behavior
            sub_behavior_index += 1
        behavior_list[behavior_index] = behavior
        behavior_index += 1
    return behavior_list

In [5]:
def tokenize(behavior, delimiter):
    behavior_vector = [3]
    prev_matchEnd = 0

    for match in re.finditer(delimiter, behavior):
        if (match.start() - prev_matchEnd) > 2: 
            behavior_vector.append(0)
        behavior_vector.append(int(match.lastgroup[1:]))
        prev_matchEnd = match.end()
    behavior_vector.append(4)
    
    return behavior_vector

In [6]:
mal_classes = ['adware', 'banking', 'riskware', 'sms']
error_hashes = []
parent_dir = 'X:\\MITRE\\MalDroid Data\\MalDroid_feature_engineering\\'

for mal_class in mal_classes:
    for sample_folder in os.listdir(parent_dir + mal_class + '\\'):
        # with open(parent_dir + mal_class + '\\' + sample_folder + '\\sample_for_analysis.apk.json') as sample_path:
        start_sample = time.time()
        with open('X:\\MITRE\\MalDroid Data\\MalDroid_feature_engineering\\riskware\\f6036a5730f8ec961bc1666cfc3269dbf9b854b1a705a65610154a697e804d79\\sample_for_analysis.apk.json') as sample_path:
            try:
                sample_behaviors = json.load(sample_path)['behaviors']['dynamic']['host']
            except:
                error_hashes.append(sample_folder)
                continue
        load_end = time.time()

        sort_start = time.time()
        sorted_behaviors = sort_behaviors(sample_behaviors)
        sort_end = time.time()
        sample_behaviors = []
        # lists are cleared after useage to preserve memory resources

        strip_start = time.time()
        stripped_behaviors = strip_unused_keys(sorted_behaviors, [])
        strip_end = time.time()
        sorted_behaviors = []

        cast_str_start = time.time()
        string_behaviors = [json.dumps(behavior) for behavior in stripped_behaviors] 
        cast_str_end = time.time()
        # check to ensure this preserves json syntax (ex "" not '' in keys)
        stripped_behaviors = []

        vector_start = time.time()
        token_times = []
        append_times = []
        vectorized_sample = [1]
        for behavior in string_behaviors:
            token_start = time.time()
            append_to_vector = tokenize(behavior, regex_pattern)
            token_end = time.time()
            token_times.append(token_end-token_start)
            append_start = time.time()
            for scalar in append_to_vector:
                vectorized_sample.append(scalar)
            append_end = time.time()
            append_times.append(append_end-append_start)
        vectorized_sample.append(2)
        vector_end = time.time()

        ds_start = time.time()
        tf_dataset = tf.data.Dataset.from_tensor_slices(vectorized_sample)
        tf_dataset = tf_dataset.map(tf.io.serialize_tensor)
        ds_end = time.time()

        write_start = time.time()
        writer = tf.data.experimental.TFRecordWriter("TFRecord_files/" + mal_class + "/" + sample_folder + ".tfrecord")
        writer.write(tf_dataset)
        end_sample = time.time()

        print("full pass: {0}".format(end_sample-start_sample))
        print("load sample: {0}".format(load_end-start_sample))
        print("sort: {0}".format(sort_end-sort_start))
        print("strip: {0}".format(strip_end-strip_start))
        print("cast: {0}".format(cast_str_end-cast_str_start))
        print("vectorize: {0}".format(vector_end-vector_start))
        print("avg tokenize: {0}".format(mean(token_times)))
        print("avg append: {0}".format(mean(append_times)))
        print("dataset: {0}".format(ds_end-ds_start))
        print("write: {0}".format(write_end-write_start))
        break
    break

0.08800029754638672
0.025996685028076172
0.15400099754333496
0.037000179290771484
0.027999401092529297
0.026000022888183594
0.027001380920410156
0.027001142501831055
0.02700018882751465
0.026999473571777344
0.02700042724609375
0.029006004333496094
0.026993274688720703
0.0280001163482666
0.026006221771240234
0.027994871139526367
0.02700185775756836
0.0279998779296875
0.027997732162475586
0.03000020980834961
0.026999473571777344
0.027000904083251953
0.027999162673950195
0.03400111198425293
0.029007911682128906
0.027991771697998047
0.02700018882751465
0.029000282287597656
0.02700018882751465
0.027998685836791992
0.030002593994140625
0.033997535705566406
0.03299975395202637
0.032001495361328125
0.030005693435668945
0.033006906509399414
0.028993844985961914
0.032998085021972656
0.029999732971191406
0.02899956703186035
0.029007911682128906
0.028995275497436523
0.027997732162475586
0.028001070022583008
0.029004573822021484
0.02899479866027832
0.0279996395111084
0.031000852584838867
0.02900481

KeyboardInterrupt: 